In [1]:
# 4/6/23
# KM Altenburger

# steps for downloading repo internally; run w/ "grasp_workflows kernel"
# sudo feature install ttls_fwdproxy
# mgt import --src-type pypi sdist
# mgt import --src-type pypi nvidia-cublas-cu11==11.10.3.66
# mgt import --src-type pypi ogb

import sys

sys.path.insert(0, "/home/kaltenburger/fbsource/ogb")
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import ogb
import pandas as pd
import torch
#import torch_geometric
from ogb.nodeproppred import NodePropPredDataset
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import auc, log_loss, precision_recall_curve, roc_auc_score
from sklearn.model_selection import train_test_split


Bad key "axes.color_cycle" on line 214 in
/home/kaltenburger/.matplotlib/matplotlibrc.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


I0411 221732.529 _utils_internal.py:179] NCCL_DEBUG env var is set to None


I0411 221732.530 _utils_internal.py:197] NCCL_DEBUG is forced to WARN from None


### read & set-up dataset


In [2]:
dataset = NodePropPredDataset(name='ogbn-proteins')
g = dataset.graph # get edgelist and graph metadata
y = dataset.labels # outcomes
node_species_df = pd.DataFrame({'node_id':range(len(g['node_species'].flatten())), 'node_species': g['node_species'].flatten()})
node_species_df.head()

,node_id,node_species
0,0,3702
1,1,3702
2,2,3702
3,3,3702
4,4,3702


In [3]:
# create edgelist
edge_list_df = pd.DataFrame(g["edge_index"].T)
edge_list_df.columns = ['src','dst']
edge_list_df.head()

,src,dst
0,1,96401
1,96401,1
2,1,68343
3,68343,1
4,1,125069


In [4]:
# sanity check on inter-species connections
edge_list_df_check = edge_list_df.merge(node_species_df, how = 'left', left_on = 'src', right_on = 'node_id')
edge_list_df_check.drop('node_id', axis = 1, inplace=True)
edge_list_df_check = edge_list_df_check.merge(node_species_df, how = 'left', left_on = 'dst', right_on = 'node_id')
edge_list_df_check.drop('node_id', axis = 1, inplace=True)
#edge_list_df_check.head()
print(np.mean(edge_list_df_check['node_species_x']==edge_list_df_check['node_species_y']))
print(np.mean(edge_list_df_check['node_species_x']!=edge_list_df_check['node_species_y']))

In [ ]:
# edge weights
edge_w = pd.DataFrame(g['edge_feat'])
w0 = np.mean(edge_w,1) # average edge weight
#w0 = edge_w[3]
edge_list_df['weight'] = w0
edge_list_df.head()

In [ ]:
plt.hist(edge_list_df.weight)
plt.show()

In [ ]:
y0 = pd.DataFrame(y)
y0 = y0.iloc[:, 0:25]  # part I
#y0 = y0.iloc[:,25:50] # part II
#y0 = y0.iloc[:,50:112] # part II
y0["node_id"] = range(len(y0[24]))

split_idx = dataset.get_idx_split()
train_idx, valid_idx, test_idx = (
    split_idx["train"],
    split_idx["valid"],
    split_idx["test"],
)

X_train = y0["node_id"].iloc[train_idx]
y_train = y0.iloc[train_idx]

X_valid = y0["node_id"].iloc[valid_idx]
X_test = y0["node_id"].iloc[test_idx]
y_valid = y0.iloc[valid_idx]
y_test = y0.iloc[test_idx]

# merge in labels for training nodes on a user's connections
relevant_edges_tmp = edge_list_df.merge(
    y_train, left_on="dst", right_on="node_id", how="left"
)

In [47]:
# set-up loop
rocauc_v2_ud_w = []


for i in range(0, 25):  # part I
    # for i in range(25, 50): # part II
    # for i in range(50,112): # part III
    print(i)
    relevant_edges = relevant_edges_tmp[["src", "dst", "weight", "node_id", i]]
    relevant_edges.columns = ["src", "dst", "weight", "node_id", "y"]
    label1 = relevant_edges.groupby(["src"])["y"].sum().reset_index()
    labeltotal = relevant_edges.groupby(["src"])["y"].count().reset_index()
    relevant_edgelist = labeltotal.merge(label1, on="src")
    relevant_edgelist["label_0"] = relevant_edgelist["y_x"] - relevant_edgelist["y_y"]
    relevant_edgelist.rename(columns={"y_y": "label_1"}, inplace=True)
    relevant_edgelist.drop(["y_x"], axis=1, inplace=True)

    ## v1 - undirected, weighted
    normalizer = pd.DataFrame(y_train[i].value_counts(normalize=True, ascending=True))

    # step 1 - one hop scores
    step1_one_hop_scores = pd.DataFrame(
        {
            "src": relevant_edgelist.src,
            "frac": np.log(
                (relevant_edgelist["label_1"] + 1) / (relevant_edgelist["label_0"] + 1)
            )
            * list((normalizer.iloc[0]) / (normalizer.iloc[1]))[0],
        }
    )

    ## v2* undirected, weighted
    W = 4
    interval = pd.DataFrame(
        relevant_edges.weight.quantile([0.25, 0.5, 0.75])
    ).reset_index()
    interval.columns = ["interval", "weight"]

    relevant_edgelist_dict = {}
    for w in range(W):
        if w == 0:
            idx = relevant_edges.weight <= interval.weight[0]
        if w == 1:
            idx = (relevant_edges.weight > interval.weight[0]) & (
                relevant_edges.weight <= interval.weight[1]
            )
        if w == 2:
            idx = (relevant_edges.weight > interval.weight[1]) & (
                relevant_edges.weight <= interval.weight[2]
            )
        if w == 3:
            idx = relevant_edges.weight > interval.weight[2]

        label1 = (
            relevant_edges.iloc[np.where(idx)[0]]
            .groupby(["src"])["y"]
            .sum()
            .reset_index()
        )
        labeltotal = (
            relevant_edges.iloc[np.where(idx)[0]]
            .groupby(["src"])["y"]
            .count()
            .reset_index()
        )

        relevant_edgelist = labeltotal.merge(label1, on="src", how="outer").fillna(0)
        relevant_edgelist["label_0"] = (
            relevant_edgelist["y_x"] - relevant_edgelist["y_y"]
        )
        relevant_edgelist.rename(columns={"y_y": "label_1"}, inplace=True)
        relevant_edgelist.drop(["y_x"], axis=1, inplace=True)
        relevant_edgelist.columns = ["src", "label_1_" + str(w), "label_0_" + str(w)]
        relevant_edgelist_dict[w] = relevant_edgelist

    merged_edgelist = relevant_edgelist_dict[0].merge(
        relevant_edgelist_dict[1], on="src", how="outer"
    )
    merged_edgelist = merged_edgelist.merge(
        relevant_edgelist_dict[2], on="src", how="outer"
    )
    merged_edgelist = merged_edgelist.merge(
        relevant_edgelist_dict[3], on="src", how="outer"
    ).fillna(0)
    normalizer = pd.DataFrame(y_train[i].value_counts(normalize=True, ascending=True))
    step1_one_hop_scores = pd.DataFrame(
        {
            "src": merged_edgelist.src,
            "frac0": np.log(
                (merged_edgelist["label_1_0"] + 1) / (merged_edgelist["label_0_0"] + 1)
            )
            * list((normalizer.iloc[0]) / (normalizer.iloc[1]))[0],
            "frac1": np.log(
                (merged_edgelist["label_1_1"] + 1) / (merged_edgelist["label_0_1"] + 1)
            )
            * list((normalizer.iloc[0] / normalizer.iloc[1]))[0],
            "frac2": np.log(
                (merged_edgelist["label_1_2"] + 1) / (merged_edgelist["label_0_2"] + 1)
            )
            * list((normalizer.iloc[0] / normalizer.iloc[1]))[0],
            "frac3": np.log(
                (merged_edgelist["label_1_3"] + 1) / (merged_edgelist["label_0_3"] + 1)
            )
            * list((normalizer.iloc[0] / normalizer.iloc[1]))[0],
        }
    )
    two_hop_stg = relevant_edges.merge(
        step1_one_hop_scores, left_on="dst", right_on="src", how="inner"
    )
    interval = pd.DataFrame(
        two_hop_stg.weight.quantile([0.25, 0.5, 0.75])
    ).reset_index()
    interval.columns = ["interval", "weight"]

    two_0 = (
        two_hop_stg[two_hop_stg.weight <= interval.weight[0]]
        .groupby(["src_x"])["frac0"]
        .sum()
        .reset_index()
    )
    # Version II*
    two_0_new1 = (
        two_hop_stg[two_hop_stg.weight <= interval.weight[0]]
        .groupby(["src_x"])["frac1"]
        .sum()
        .reset_index()
    )
    two_0_new2 = (
        two_hop_stg[two_hop_stg.weight <= interval.weight[0]]
        .groupby(["src_x"])["frac2"]
        .sum()
        .reset_index()
    )
    two_0_new3 = (
        two_hop_stg[two_hop_stg.weight <= interval.weight[0]]
        .groupby(["src_x"])["frac3"]
        .sum()
        .reset_index()
    )

    two_1 = (
        two_hop_stg[
            (two_hop_stg.weight > interval.weight[0])
            & (two_hop_stg.weight <= interval.weight[1])
        ]
        .groupby(["src_x"])["frac1"]
        .sum()
        .reset_index()
    )
    # Version II*
    two_1_new1 = (
        two_hop_stg[
            (two_hop_stg.weight > interval.weight[0])
            & (two_hop_stg.weight <= interval.weight[1])
        ]
        .groupby(["src_x"])["frac0"]
        .sum()
        .reset_index()
    )
    two_1_new2 = (
        two_hop_stg[
            (two_hop_stg.weight > interval.weight[0])
            & (two_hop_stg.weight <= interval.weight[1])
        ]
        .groupby(["src_x"])["frac2"]
        .sum()
        .reset_index()
    )
    two_1_new3 = (
        two_hop_stg[
            (two_hop_stg.weight > interval.weight[0])
            & (two_hop_stg.weight <= interval.weight[1])
        ]
        .groupby(["src_x"])["frac3"]
        .sum()
        .reset_index()
    )

    two_2 = (
        two_hop_stg[
            (two_hop_stg.weight > interval.weight[1])
            & (two_hop_stg.weight <= interval.weight[2])
        ]
        .groupby(["src_x"])["frac2"]
        .sum()
        .reset_index()
    )

    # Version II*
    two_2_new1 = (
        two_hop_stg[
            (two_hop_stg.weight > interval.weight[1])
            & (two_hop_stg.weight <= interval.weight[2])
        ]
        .groupby(["src_x"])["frac0"]
        .sum()
        .reset_index()
    )
    two_2_new2 = (
        two_hop_stg[
            (two_hop_stg.weight > interval.weight[1])
            & (two_hop_stg.weight <= interval.weight[2])
        ]
        .groupby(["src_x"])["frac1"]
        .sum()
        .reset_index()
    )
    two_2_new3 = (
        two_hop_stg[
            (two_hop_stg.weight > interval.weight[1])
            & (two_hop_stg.weight <= interval.weight[2])
        ]
        .groupby(["src_x"])["frac3"]
        .sum()
        .reset_index()
    )

    two_3 = (
        two_hop_stg[(two_hop_stg.weight > interval.weight[2])]
        .groupby(["src_x"])["frac3"]
        .sum()
        .reset_index()
    )

    # Version II*
    two_3_new1 = (
        two_hop_stg[(two_hop_stg.weight > interval.weight[2])]
        .groupby(["src_x"])["frac0"]
        .sum()
        .reset_index()
    )
    two_3_new2 = (
        two_hop_stg[(two_hop_stg.weight > interval.weight[2])]
        .groupby(["src_x"])["frac1"]
        .sum()
        .reset_index()
    )
    two_3_new3 = (
        two_hop_stg[(two_hop_stg.weight > interval.weight[2])]
        .groupby(["src_x"])["frac2"]
        .sum()
        .reset_index()
    )

    two_hop_v2 = two_0.merge(two_1, how="outer", on="src_x").fillna(0)
    two_hop_v2 = two_hop_v2.merge(two_2, how="outer", on="src_x").fillna(0)
    two_hop_v2 = two_hop_v2.merge(two_3, how="outer", on="src_x").fillna(0)

    # Version II*
    two_hop_v2 = two_hop_v2.merge(two_1_new1, how="outer", on="src_x").fillna(0)
    two_hop_v2 = two_hop_v2.merge(two_1_new2, how="outer", on="src_x").fillna(0)
    two_hop_v2 = two_hop_v2.merge(two_1_new3, how="outer", on="src_x").fillna(0)
    two_hop_v2 = two_hop_v2.merge(two_2_new1, how="outer", on="src_x").fillna(0)
    two_hop_v2 = two_hop_v2.merge(two_2_new2, how="outer", on="src_x").fillna(0)
    two_hop_v2 = two_hop_v2.merge(two_2_new3, how="outer", on="src_x").fillna(0)
    two_hop_v2 = two_hop_v2.merge(two_3_new1, how="outer", on="src_x").fillna(0)
    two_hop_v2 = two_hop_v2.merge(two_3_new2, how="outer", on="src_x").fillna(0)
    two_hop_v2 = two_hop_v2.merge(two_3_new3, how="outer", on="src_x").fillna(0)
    two_hop_v2 = two_hop_v2.merge(two_0_new1, how="outer", on="src_x").fillna(0)
    two_hop_v2 = two_hop_v2.merge(two_0_new2, how="outer", on="src_x").fillna(0)
    two_hop_v2 = two_hop_v2.merge(two_0_new3, how="outer", on="src_x").fillna(0)

    two_hop_v2.columns = [
        "src_x",
        "frac0",
        "frac1",
        "frac2",
        "frac3",
        "frac4",
        "frac5",
        "frac6",
        "frac7",
        "frac8",
        "frac9",
        "frac10",
        "frac11",
        "frac12",
        "frac13",
        "frac14",
        "frac15",
    ]
    two_hop_v2_valid = two_hop_v2.merge(
        y_valid, left_on="src_x", right_on="node_id", how="right"
    )

    pred_v2 = y_test.merge(two_hop_v2, right_on="src_x", left_on="node_id", how="left")
    clf = LogisticRegression(random_state=0).fit(
        two_hop_v2_valid[
            [
                "frac0",
                "frac1",
                "frac2",
                "frac3",
                "frac4",
                "frac5",
                "frac6",
                "frac7",
                "frac8",
                "frac9",
                "frac10",
                "frac11",
                "frac12",
                "frac13",
                "frac14",
                "frac15",
            ]
        ],
        two_hop_v2_valid[i],
    )

    print(
        "roc-auc, v2: ",
        roc_auc_score(
            pred_v2[i],
            clf.predict_proba(
                pred_v2[
                    [
                        "frac0",
                        "frac1",
                        "frac2",
                        "frac3",
                        "frac4",
                        "frac5",
                        "frac6",
                        "frac7",
                        "frac8",
                        "frac9",
                        "frac10",
                        "frac11",
                        "frac12",
                        "frac13",
                        "frac14",
                        "frac15",
                    ]
                ]
            )[:, 1],
        ),
    )

    rocauc_v2_ud_w.append(
        roc_auc_score(
            pred_v2[i],
            clf.predict_proba(
                pred_v2[
                    [
                        "frac0",
                        "frac1",
                        "frac2",
                        "frac3",
                        "frac4",
                        "frac5",
                        "frac6",
                        "frac7",
                        "frac8",
                        "frac9",
                        "frac10",
                        "frac11",
                        "frac12",
                        "frac13",
                        "frac14",
                        "frac15",
                    ]
                ]
            )[:, 1],
        )
    )

    print("************************************************************************")

0


roc-auc, v2:  0.6502859184753726
baseline, v2:  0.08468738603671137 24679
************************************************************************
1


roc-auc, v2:  0.6538896803615811
baseline, v2:  0.08464686575631103 24679
************************************************************************
2


roc-auc, v2:  0.6792864866922101
baseline, v2:  0.06357631994813404 24679
************************************************************************
3


roc-auc, v2:  0.7397756910706484
baseline, v2:  0.057660359009684345 24679
************************************************************************
4


...

roc-auc, v2:  0.6543830374004536
baseline, v2:  0.021151586368977675 24679
************************************************************************
21


roc-auc, v2:  0.7687187207077459
baseline, v2:  0.03201102151626889 24679
************************************************************************
22


roc-auc, v2:  0.7526713465476323
baseline, v2:  0.030025527776652215 24679
************************************************************************
23


roc-auc, v2:  0.7481672556841693
baseline, v2:  0.029620324972648812 24679
************************************************************************
24


roc-auc, v2:  0.6596957202662759
baseline, v2:  0.029134081607844725 24679
************************************************************************


In [49]:
with open('rocauc_v2_ud_w_part1.npy', 'wb') as f:
#with open('rocauc_v2_ud_w_part2.npy', 'wb') as f:
#with open('rocauc_v2_ud_w_part3.npy', 'wb') as f:
    np.save(f, rocauc_v2_ud_w)